In [2]:
from pathlib import Path
import pandas as pd

DATA_DIR = Path("../data")
products = pd.read_csv(DATA_DIR/"sample_products.csv")
def tags_to_text(tag_string):
    if pd.isna(tag_string) or str(tag_string).strip()=="":
        return ""
    return " ".join(str(tag_string).split("|"))

products["tags_text"] = products["tags"].apply(tags_to_text)

products["text_features"] = (
    products["category"].fillna("") + " " +
    products["brand"].fillna("")+ " " +
    products["tags_text"].fillna("")+ " " +
    products["description"].fillna("")
)

products[["product_id","category", "brand", "tags_text", "text_features"]].head()

,product_id,category,brand,tags_text,text_features
0,1,Electronics,PageTurner,battery wireless bluetooth 4K,Electronics PageTurner battery wireless blueto...
1,2,Beauty,PageTurner,anti-aging SPF vitamin-c sensitive skincare,Beauty PageTurner anti-aging SPF vitamin-c sen...
2,3,Fashion,PageTurner,summer formal casual leather,Fashion PageTurner summer formal casual leathe...
3,4,Home,Silk&Co,eco-friendly durable minimal,Home Silk&Co eco-friendly durable minimal Silk...
4,5,Beauty,PageTurner,SPF hydrating anti-aging skincare vitamin-c,Beauty PageTurner SPF hydrating anti-aging ski...
